In [1]:
%pip install bayesian-optimization
%pip install livelossplot
%pip install lightgbm
%pip install shap

import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import numpy as np
import os, contextlib, sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import linear_model
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.preprocessing import quantile_transform, StandardScaler, RobustScaler
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold, cross_val_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.neural_network import MLPRegressor
from livelossplot import PlotLosses
import xgboost as xgb
import lightgbm as lgb
import datetime
import shap
import numpy as np
from scipy import stats
from itertools import combinations
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os, contextlib, sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition
from sklearn import datasets


%cd /content/drive/My\ Drive/Colab\ Notebooks

np.random.seed(seed=1)

/content/drive/My Drive/Colab Notebooks


pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [0]:
gpu = torch.device("cuda:0")

In [0]:
def loaddata(directory, t=True):
  
    feature_list = list()
    train_data_list = list()
    train_target_list = list()
    name = list()


    for i,filename in enumerate(os.listdir(directory)):
        data_matrix = np.genfromtxt(f'{directory}/{filename}')
        if filename == 'Data2scaled.txt': 
            name.append(filename)
            raw_data = np.genfromtxt('data/etalon_etalon_29May20.ccfSum-telemetry.csv',delimiter=',',names=True)
            feature_list.append(list(raw_data.dtype.names))
            data_matrix = np.delete(data_matrix, 2, 1)
            test_data = data_matrix[:,[i for i in range(len(data_matrix[1])) if i!=1]]
            test_target = data_matrix[:,1]
        else:
            data_matrix = np.delete(data_matrix, 2, 1)
            train_data_list.append(data_matrix[:,[i for i in range(len(data_matrix[1])) if i!=1]])
            train_target_list.append(data_matrix[:,1])
    feature_list = feature_list[0]
    feature_list.remove('RV')
    feature_list.remove('RV_ERR')    


    if t == True:
      train_data_list = [quantile_transform(data,n_quantiles=100,copy=True) for data in train_data_list]
      test_data = quantile_transform(test_data,n_quantiles=100,copy=True)
      train_data_torch = [torch.from_numpy(data).unsqueeze(dim=1).float().to(gpu) for data in train_data_list]
      train_target_torch = [torch.from_numpy(data).unsqueeze(dim=1).float().to(gpu) for data in train_target_list]
      test_data_torch, test_target_torch = torch.from_numpy(np.array(test_data)).unsqueeze(dim=1).float().to(gpu), torch.from_numpy(np.array(test_target)).float().to(gpu)
      return train_data_torch, train_target_torch, test_data_torch, test_target_torch, np.array(feature_list,dtype=str)

    
    elif t == False:
      train_data_list = [quantile_transform(data,n_quantiles=100,copy=True) for data in train_data_list]
      test_data = quantile_transform(test_data,n_quantiles=100,copy=True)
      train_data = np.concatenate(train_data_list, axis=0)
      train_target = np.concatenate(train_target_list, axis=0)
      train_data = np.delete(train_data, 0, 1)
      test_data = np.delete(test_data, 0, 1)
      return train_data, train_target, test_data, test_target, name[0]

**Loading in data:** For gpu (torch-based, t=True) and cpu (sklearn-based, t=False)

In [0]:
train_data_torch, train_target_torch, test_data_torch, test_target_torch, feature_list = loaddata('Scaleddata', t=True)

In [0]:
train_data_numpy, train_target_numpy, test_data_numpy, test_target_numpy, name = loaddata('Scaleddata', t=False)

In [0]:
class LSTMTagger(nn.Module):

    def __init__(self, inputsize, layers, hiddensize, drop_out):
        super(LSTMTagger, self).__init__()
        self.inputsize = inputsize
        self.hiddensize = hiddensize
        self.layers = layers
        self.drop_out = drop_out

        self.lstm = nn.LSTM(input_size=self.inputsize, hidden_size = self.hiddensize, num_layers=layers, dropout=drop_out)
        self.hidden2radial = nn.Linear(in_features=hiddensize, out_features=1)

    def forward(self, x):
        x, _ = self.lstm(x)
        #x = F.elu(x)
        x = self.hidden2radial(x)
        return x

**Bayesian Optimization:** A generalized version to optimize both torch based and sklearn based algorithms

In [0]:
def val(inputsize: int, algorithm, data, targets, **params):
    param_list = list()
    for arg in params.values():
      param_list.append(arg)

    if algorithm == LSTMTagger:
      estimator = algorithm(inputsize, int(param_list[2]), int(param_list[1]), param_list[0])
      optimizer = torch.optim.Adam(estimator.parameters(),lr=param_list[3])
      judge = list()
      estimator = estimator.to(gpu)
      criterion = nn.MSELoss()
      for i,valdata in enumerate(train_data_torch):
          traindata = train_data_torch[:i] + train_data_torch[i+1:]
          traintarget = train_target_torch[:i] + train_target_torch[i+1:]
          valtarget = train_target_torch[i]
          judge_list = list()
          n_epochs = 100
          for e in range(n_epochs):
            estimator.train()
            epoch_losses = list()
            epoch_evs = list()
            acc_list = list()
            loss_list = list()
            for batch in range(len(traindata)):
              estimator.zero_grad()
              optimizer.zero_grad() 
              prediction = estimator(traindata[batch])
              target = traintarget[batch]
              # Calculating the loss function
              loss = criterion(prediction.squeeze(dim=2), target)
              # Calculating the gradient
              loss.backward()
              optimizer.step()
            with torch.no_grad():
              estimator.eval()
              train_prediction = estimator(valdata).squeeze(dim=1)
              loss_list.append( float(criterion((train_prediction),valtarget).detach().cpu()) )
              acc_list.append( explained_variance_score(valtarget.cpu(), train_prediction.cpu()) )
            judge_list.append(np.mean(acc_list))
      return np.mean(judge_list)

    else:
      for key in params.keys():
        if params[key] >= 1:
          params[key] = int(params[key])
      estimator = algorithm(random_state=27, **params)
      cval = cross_val_score(estimator, data, targets, cv=5,scoring='explained_variance')
      return cval.mean()

def optimize(algorithm, data, targets, params, n_iter):
    def crossval_wrapper(data=data, targets=targets, **params):
        return val(inputsize = 26,
                   algorithm = algorithm, 
                   data = data, 
                   targets = targets, 
                   **params)

    optimizer = BayesianOptimization(f=crossval_wrapper, 
                                     pbounds=params, 
                                     random_state=27, 
                                     verbose=2)
    optimizer.maximize(init_points=2, n_iter=n_iter)

    return optimizer

In [0]:
model_list = list()

**Long short term memory:**

In [0]:
params_LSTM = {"layers": (1, 3),
               "hiddensize": (100, 400),
               "drop_out": (0.2, 0.6),
               "lr": (0.0001, 0.00001)}

BO_LSTM = optimize(algorithm = LSTMTagger, data = train_data_torch, targets = train_target_torch, params = params_LSTM, n_iter = 6)

print(BO_LSTM.max)

max_params_LSTM = BO_LSTM.max['params']

|   iter    |  target   | drop_out  | hidden... |  layers   |    lr     |
-------------------------------------------------------------------------
|  1        |  0.7485   |  0.3703   |  344.4    |  2.471    |  2.188e-0 |
|  2        |  0.8797   |  0.3534   |  393.8    |  2.786    |  8.113e-0 |
|  3        |  0.5813   |  0.2306   |  100.0    |  2.132    |  1e-05    |


dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4854891532706884 and num_layers=1


|  4        |  0.7117   |  0.4855   |  400.0    |  1.257    |  1e-05    |


In [0]:
criterion = nn.MSELoss()
judge = list()
for i,valdata in enumerate(train_data_torch):
      model = LSTMTagger(26,layers=int(max_params_LSTM['layers']), hiddensize=int(max_params_LSTM['hiddensize']),drop_out=max_params_LSTM['drop_out']).to(gpu)
      optimizer = optim.Adam(model.parameters(),lr=max_params_LSTM['lr'])

      traindata = train_data_torch[:i] + train_data_torch[i+1:]
      traintarget = train_target_torch[:i] + train_target_torch[i+1:]
      valtarget = train_target_torch[i]
      judge_list = list()

      plotlosses = PlotLosses()
      n_epochs = 100
      for e in range(n_epochs):
          model.train()
          epoch_losses = list()
          epoch_evs = list()
          acc_list = list()
          loss_list = list()
          for batch in range(len(traindata)):
              model.zero_grad()
              optimizer.zero_grad() 
              prediction = model(traindata[batch])
              target = traintarget[batch]
              # Calculating the loss function
              loss = criterion(prediction.squeeze(dim=2), target)
              epoch_losses.append(float(loss))
              evs = explained_variance_score(target.squeeze(dim=1).detach().cpu().numpy(),prediction.squeeze(dim=1).detach().cpu().numpy())
              epoch_evs.append(evs)
              # Calculating the gradient
              loss.backward()
              optimizer.step()
          with torch.no_grad():
              model.eval()
              train_prediction = model(valdata).squeeze(dim=1)
              loss_list.append( float(criterion((train_prediction),valtarget).detach().cpu()) )
              acc_list.append( explained_variance_score(valtarget.cpu(), train_prediction.cpu()) )
          print(e, np.mean(epoch_losses), np.mean(epoch_evs))
          judge_list.append(np.mean(acc_list))
      judge.append([np.mean(judge_list),model])
sorted(judge, key=lambda x: x[0])
winner = judge[0]
model_list.append(winner[1])

**Light GBM:**

In [0]:
params_LGBM = {'num_leaves': (2, 3),
               'max_depth': (70, 200),
               'learning_rate': (0.00001, 0.01)}

BO_LGBM = optimize(algorithm = lgb.LGBMRegressor, data = train_data_numpy, targets = train_target_numpy, params = params_LGBM, n_iter=30)

print(BO_LGBM.max)

max_params_LGBM = BO_LGBM.max['params']

model_LGBM = lgb.LGBMRegressor(num_leaves = int(max_params_LGBM['num_leaves']), 
                               max_depth= int(max_params_LGBM['max_depth']), 
                               learning_rate = max_params_LGBM['learning_rate']
                               )


model_LGBM.fit(train_data_numpy, train_target_numpy)



model_list.append(model_LGBM)

**XG Boost:**

In [0]:
params_XGB = {"eta": (0.1, 0.5),
               "max_depth": (1, 10),
               "num_round": (1, 40),
              "n_estimators": (2, 10)}

BO_XGB = optimize(algorithm = xgb.XGBRegressor, data = train_data_numpy, targets = train_target_numpy, params = params_XGB, n_iter=30)

print(BO_XGB.max)

max_params_XGB = BO_XGB.max['params']

model_XGB = xgb.XGBRegressor(max_depth = int(max_params_XGB['max_depth']), 
                             num_round = int(max_params_XGB['num_round']),
                             eta = max_params_XGB['eta'],
                             n_estimators = int(max_params_XGB['n_estimators'])
                             )     
     

model_XGB.fit(train_data_numpy, train_target_numpy)     

model_list.append(model_XGB)

**Ordinary Least Squares:** No need for Bayesian Optimization \\
**Multi-Layer Perception:**

In [0]:
params_MLP = {"hidden_layer_sizes": (50, 300),
               "alpha": (0, 0.5),
               "learning_rate_init": (0.0001, 0.5),}

BO_MLP = optimize(algorithm = MLPRegressor, data = train_data_numpy, targets = train_target_numpy, params = params_MLP, n_iter=30)

print(BO_MLP.max)

max_params_MLP = BO_MLP.max['params']

model_MLP = MLPRegressor(hidden_layer_sizes= int(max_params_MLP['hidden_layer_sizes']),
                         alpha = max_params_MLP['alpha'],
                         learning_rate_init= max_params_MLP['learning_rate_init'],
                         )
model_MLP.fit(train_data_numpy, train_target_numpy)

model_list.append(model_MLP)

**Ordinary Least Squares:**

In [0]:
model_OLS = linear_model.LinearRegression()
model_OLS.fit(train_data_numpy, train_target_numpy)
model_list.append(model_OLS)

**Plot:**

In [0]:
time = test_data_torch.cpu().squeeze().detach().numpy()[:,0]

fig = plt.figure('All models')
colorlist = ['#1b9e77', '#d95f02', '#7570b3', '#e7298a', '#66a61e', '#e6ab02']


plt.plot(time, test_target_numpy, linestyle='-', linewidth=4, label='True', color='k')
for i,model in enumerate(model_list):
  if i == 0:
    prediction = model(test_data_torch).squeeze(dim=1).detach().cpu().numpy()
    plt.plot(time, prediction, linestyle='--',label=f'{model}'.split('(')[0].replace('Tagger',''), color=colorlist[i],linewidth=2)
  elif i == len(model_list)-1:
    plt.plot(time, model.predict(test_data_numpy),linestyle='--',label=f'{model}'.split('(')[0].replace('Regression',''), color=colorlist[i], linewidth=2)
  else:
    plt.plot(time, model.predict(test_data_numpy),linestyle='--',label=f'{model}'.split('(')[0].replace('Regressor',''), color=colorlist[i], linewidth=2)
plt.legend(loc='best')
plt.title(f"Testing on {name.replace('scaled.txt','')}")
plt.xlabel('Scaled time')
plt.ylabel('Scaled drift')
plt.style.use('seaborn-white')
plt.style.use('seaborn-ticks')
fig.patch.set_facecolor('#f2f2f2')
cur_time = str(datetime.datetime.now())
cur_time = cur_time.split('.')[0]
cur_time = cur_time.replace(' ','')
cur_time = cur_time.replace('-','')
cur_time = cur_time.replace(':','')

fig.savefig(f"figs/TotalTest{name.replace('scaled.txt','')}_{cur_time}.png", facecolor=fig.get_facecolor(), format='png', dpi=1200)

In [0]:
import pickle
for i,model in enumerate(model_list):
  filename = f'models/finalized_model_{i}.sav'
  pickle.dump(model, open(filename, 'wb'))

In [0]:
new_model_list = list()
filename = f'models/finalized_model_{i}.sav'
for i,filename in enumerate(os.listdir('models')):
  new_model_list.append(pickle.load(open(f"models/{filename}", 'rb')))



all_data = np.vstack([train_data_numpy, test_data_numpy])
feature_list = np.delete(feature_list, np.argwhere( feature_list == 'JD_UTC'),0)

for i, model in enumerate(new_model_list):
  if i == 0:
    print('Not on SHAP :(')
    continue
    #for k in range(len(train_data_torch)):
      #explainer = shap.GradientExplainer(model, data= train_data_torch[k])
      #modelname = f'{model}'.split('(')[0].replace('Regressor','')
      #modelname = f'{modelname}_{k}'
      #model.train()
      #shap_values = explainer.shap_values(train_data_torch[k])
      #shap.summary_plot(shap_values, train_data_torch[k], 
      #                plot_type="bar",show=False, feature_names=feature_list,
      #                color="#4d9221")
    
  elif i == len(model_list)-1:
    explainer = shap.LinearExplainer(model, data = all_data)
    modelname = f'{model}'.split('(')[0].replace('Regression','')
    shap_values = explainer.shap_values(all_data)
    shap.summary_plot(shap_values, all_data, plot_type = "bar", 
                      show=False, feature_names=feature_list,
                      color="#e7298a")
  elif i == 3:
    print('Not on SHAP :(')
    continue
  else:
    print(model, '\n')
    explainer = shap.TreeExplainer(model)
    modelname = f'{model}'.split('(')[0].replace('Regressor','')
    shap_values = explainer.shap_values(all_data)
    shap.summary_plot(shap_values, all_data, plot_type = "bar", 
                      show=False, feature_names=feature_list,
                      color="#e7298a")
  path = f"figs/SHAP{name.replace('scaled.txt','')}_{modelname}_{cur_time}.png"
  pl.figure(figsize=(10,50))
  pl.tight_layout()
  #pl.xlabel('Scaled time')
  #pl.ylabel('Scaled drift')
  pl.style.use('seaborn-white')
  pl.style.use('seaborn-ticks')

  pl.savefig(path, format='png', dpi=1200, facecolor=fig.get_facecolor())


In [0]:
print(shap_values.mean(0))

In [0]:
  #Taken from https://zhiyzuo.github.io/Pearson-Correlation-CI-in-Python/

#plt.rcParams.update({'font.size': 4})

def pearsonr_ci(x, y, alpha=0.05, _print=True):
	''' calculate Pearson correlation along with the confidence interval using scipy and numpy
	Parameters
	----------
	x, y : iterable object such as a list or np.array
	Input for correlation calculation
	alpha : float
	Significance level. 0.05 by default
	Returns
	-------
	r : float
	Pearson's correlation coefficient
	pval : float
	The corresponding p value
	lo, hi : float
	The lower and upper bound of confidence intervals
	'''

	r, p = stats.pearsonr(x,y)
	r_z = np.arctanh(r)
	se = 1/np.sqrt(x.size-3)
	z = stats.norm.ppf(1-alpha/2)
	lo_z, hi_z = r_z-z*se, r_z+z*se
	lo, hi = np.tanh((lo_z, hi_z))
	if _print: print(f"\t corr {r:.3f} in [{lo:.3f}, {hi:.3f}], with p={p:.3f}")
	return lo, hi, p

def corrfunc(x, y, **kws):
	r, _ = stats.pearsonr(x, y)
	ax = plt.gca()
	if not np.isnan(r): ax.annotate("r = {:.2f} € [{:.2f},{:.2f}], p = {:.2f} ".format(r, *pearsonr_ci(x,y,_print=False)), xy=(.1, .9), xycoords=ax.transAxes)

def meanfunc(x, **kws):
	m, s = x.mean(), x.std()
	ax = plt.gca()
	ax.annotate(f"{m:.2f}±{s:.2f}", xy=(.1, .9), xycoords=ax.transAxes)

def summary_analyze(data: pd.DataFrame, filename):
	print('shape', data.shape)
	# Mean and std...
	print(data.describe())

	# Covariance matrix
	for pair in combinations(data.columns, r=2):
		print(f"{pair[0]} and {pair[1]}")
		pearsonr_ci(data[pair[0]], data[pair[1]])
	# print(data.corr())
	# plt.matshow(data.corr())
	# plt.show()

	# Joint and marginal
	plt.figure(figsize=(50,50))
	cross_values = ['FIES_INSIDE_BLACK_BOX_AT_REAR',
					'FIES_INSIDE_BLACK_BOX_CENTRE',
					'FIES_ABOVE_HEATER_RADIATOR',
					'FIES_GRATING_TANK_HOUSING',
					'FIES_INSIDE_WHITE_BOX_REAR',
					'FIES_INSIDE_WHITE_BOX_CENTRE',
					'FIES_FRONT_ROOM']
	g = sb.PairGrid(data, palette=["red"],x_vars=cross_values, y_vars=cross_values)
	g.map_diag(sb.distplot, kde=False, bins=10, color='#4d9221')
	g.map_diag(meanfunc)
	g.map_upper(sb.kdeplot, cmap="Greens_d")

	g.map_lower(plt.scatter, s=10, color='#4d9221')
	g.map_lower(corrfunc)
  
	plt.style.use('seaborn-white')
	plt.style.use('seaborn-ticks')
	filename = filename.replace('scaled.txt','')
	plt.savefig(f"figs/Cross_telemetry_{filename}.png", format='png', dpi=1200, facecolor='#f2f2f2')

In [0]:
names = pd.read_csv('data/etalon_jitter_16Mar20_etalon.ccfSum-telemetry.csv')

names = list(names.columns.values.tolist()) 

dataframe = pd.read_csv(f'data/Data12345scaled.txt', sep=" ", header=None, names=names)

In [0]:
summary_analyze(dataframe, 'Data12345scaled.txt')